<a href="https://colab.research.google.com/github/Sirabhop/Preclinical-AD-EEG-classification/blob/master/Machine_Learning_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import os

In [0]:
#Set working directory and import cookbook
os.chdir("/content/drive/My Drive/Colab Notebooks/")

#Download eeg feature extraction function
%run EEG_Feature_Extraction.ipynb

# **Import Data**

In [0]:
drive.mount('/content/drive')

In [0]:
os.chdir("/content/drive/My Drive/EEG Data")

In [0]:
#Compute EEG feature
subjectID_fif = ['SS1', 'SS2', 'SS3']
df = getDataFrame_fif(subjectID_fif)

subjectID_bdf = ['SS4', 'SS5', 'SS6', 'SS7', 'SS8']
df2 = getDataFrame_bdf(subjectID_bdf)

df = df.append(df2, ignore_index = True)

#Export EEG feature
df.to_csv('EEG_features.csv')

In [0]:
#Import FCSRT & MoCA & Demographic data
FCSRT = pd.read_csv("FCSRT.csv")
FCSRT.drop('id', axis = 1, inplace = True)
MoCA = pd.read_csv("MoCA.csv", usecols = ['SUM'])
EEG = pd.read_csv("EEG_features.csv")

df_original = pd.concat([EEG,FCSRT,MoCA], axis = 1)
df_original.drop(['Unnamed: 0'], inplace = True, axis = 1)
df_original.rename(columns = {'SUM':'MoCA'})
df_original.drop('id', axis = 1, inplace=True)

# **Data Preprocessing**

#### **1) Scaling**

In [0]:
from sklearn import preprocessing

In [56]:
#Min-Max Scaling

scaler = preprocessing.MinMaxScaler()
df_original[df_original.columns] = scaler.fit_transform(df_original[df_original.columns])

df_original

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:375: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:376: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


,PSD_Delta,PSD_Theta,PSD_Alpha,PSD_Beta,PSD_Gamma,MSF,SE,AC,wSMI_Alpha,wSMI_Theta,IR-FreeR,IR-CuedR,DL-FreeR,DL-CuedR,SUM
0,NaN,0.003081,0.003081,0.011514,0.078478,0.032240,1.000000,0.000000,0.000000,0.000000,0.0,1.00,0.2,0.333333,0.9375
1,NaN,0.000267,0.000267,0.002817,0.000000,0.227158,0.654121,0.302423,0.349803,0.328528,0.2,0.25,0.6,0.333333,0.0625
2,NaN,0.000495,0.000495,0.024244,0.346150,0.569739,0.724031,0.972203,0.531520,0.528180,1.0,0.00,1.0,0.000000,0.1875
3,NaN,1.000000,1.000000,1.000000,1.000000,0.381145,0.654121,0.253809,0.682812,0.678925,1.0,0.00,1.0,0.333333,1.0000
4,NaN,0.000302,0.000302,0.003871,0.006691,0.294278,0.000000,0.680209,0.795899,0.797631,0.8,0.00,0.2,1.000000,0.3750
5,NaN,0.000000,0.000000,0.000000,0.004384,0.000000,0.000000,0.400103,0.884533,0.892868,0.0,0.00,0.0,0.000000,0.5625
6,NaN,0.001848,0.001848,0.016655,0.179131,1.000000,0.801038,0.483071,0.946392,0.950359,0.4,0.75,0.6,0.666667,0.8750
7,NaN,0.000046,0.000046,0.004230,0.045855,0.225229,0.000000,1.000000,1.000000,1.000000,0.8,0.25,0.6,1.000000,0.0000


#### **2) Generate Data**

The section below, I've random the value of each feature in order to demonstrate ML model because the situation of *COVID-19* that I cannot continue run the experiment as expected.

In [0]:
for i in range(7, 60):
  

In [0]:
df_original